In [3]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

In [8]:
# load combined RPE_PR models
from pathlib import Path
from cobra.io import read_sbml_model

mod_RPE_PR = read_sbml_model(Path().cwd() / "models" / "mod_RPE_PR.xml")
mod_RPE_PR.id = 'RPE_PR'
mod_Human1_Human1 = read_sbml_model(Path().cwd() / "models" /  "mod_Human1_Human1.xml")
mod = read_sbml_model(Path().cwd() / 'models/Human-GEM.xml')
mod_PR = read_sbml_model(Path().cwd() / 'models/cs_mods/model_13_MeanExpression_RodPhotoreceptors__LiangEtAl_2019_.xml')
mod_RPE = read_sbml_model(Path().cwd() / 'models/cs_mods/model_6_MeanExpression_RPE__VoigtEtAl2019_.xml')

In [9]:
# modify models
# load blood exchange bounds
import pandas as pd
df = pd.read_excel(Path().cwd() / 'rxn_bounds/blood_selection.xlsx') # with col names 'id' 'lb' 'ub'

# set objective function (PR ATP DM)
mod_RPE_PR.objective = 'MAR03964_PR'
mod_Human1_Human1.objective = 'MAR03964_PR'
mod.objective = 'MAR03964'
mod_PR.objective = 'MAR03964'
mod_RPE.objective = 'MAR03964'

ValueError: invalid objective

In [ ]:
def open_blood_exchange(mod,br, glucose_level):  
    br_updated = br.copy()
    br_updated.update((x, y * glucose_level) for x, y in br_updated.items())
    for key in br:
        mod.reactions.get_by_id(key).bounds = (br_updated[key],1000)# allow influx for RPE exchange reactions (blood exchange)
        
def close_blood_exchange(mod,br):  
    for key in br:
        mod.reactions.get_by_id(key).bounds = (0,1000)# allow influx for RPE exchange reactions (blood exchange)

In [ ]:
blood_ratio = {df['id'][i]:df['ratio [glucose]/[x]'] [i] for i in range(len(df['id']))}
glucose_level = -2.622342019
# blood_ratio.update((x, y*glucose_level) for x, y in blood_ratio.items())
blood_ratio

In [ ]:
# modify combined models (blood exchange)
for m in [mod_RPE_PR, mod_Human1_Human1]:
    m.objective = 'MAR03964_PR' # ATP demand in PR
    m.remove_reactions([r.id for r in  m.reactions if len(r.products)==0 if 'PR' in r.id]) # remove PR exchange reactions
    m.remove_metabolites([m for m in m.metabolites if len(m.reactions) == 0])              # prune metabolites that are unused
    for r in [r for r in m.reactions if len(r.products) == 0 if '_RPE' in r.id]:           # open efflux for RPE exchange reactions
        r.bounds=(0,1000) # close all exchange reactions
    open_blood_exchange(m,blood_ratio,glucose_level)# allow influx for RPE exchange reactions (blood exchange)

In [ ]:
df['id'][i]

In [ ]:
# modify single models (blood exchange)
for m in [mod_RPE_PR, mod_Human1_Human1]:
    m.objective = 'MAR03964' # ATP demand in PR
    open_blood_exchange(m,blood_ratio,glucose_level)# allow influx for RPE exchange reactions (blood exchange)

In [6]:
from src.analysis import create_permutation_dicts
# MAR09048_RPE = oxygen exchange
# MAR09034_RPE = glucose exchange
# MAR09135_RPE = lactate exchange
# MAR04896_PR_RPE =  O2[c_PR] <=> O2[e_RPE_PR]

dark_old_labels = create_permutation_dicts({'oxygen':['limited oxygen (2.2)'],\
                      'glucose':['limited glucose (2.6)'],\
                      'lactate':['lactate efflux > 4.53'],\
                      'oxygen RPE -> PR':['fixed (1.9)','unlimited']})
dark_old_estimates = create_permutation_dicts({'MAR09048_RPE':[(-2.2,1000)],\
                      'MAR09034_RPE':[(-2.62,1000)],\
                      'MAR09135_RPE':[(4.53,1000)],\
                      'MAR04896_PR_RPE':[(-1.9,-1.9),(-1000,1000)]})

dark_new_labels = create_permutation_dicts({'oxygen':['limited oxygen (3.51)'],\
                      'glucose':['limited glucose (4.18)'],\
                      'lactate':[ 'lactate efflux > 7.25'],\
                      'oxygen RPE -> PR':['fixed (3.21)','unlimited']})
dark_new_estimates = create_permutation_dicts({'MAR09048_RPE':[(-3.51,1000)],\
                      'MAR09034_RPE':[(-4.18,1000)],\
                      'MAR09135_RPE':[(7.25,1000)],\
                      'MAR04896_PR_RPE':[(-3.21,-3.21),(-1000,1000)]})

light_old_labels = create_permutation_dicts({'oxygen':['limited oxygen (1.84)'],\
                      'glucose':['limited glucose (1.37)'],\
                      'lactate':['lactate efflux > 2.81'],\
                      'oxygen RPE -> PR':['fixed (1.59)','unlimited']})
light_old_estimates = create_permutation_dicts({'MAR09048_RPE':[(-1.84,1000)],\
                      'MAR09034_RPE':[ (-1.37,1000)],\
                      'MAR09135_RPE':[ (2.81,1000)],\
                      'MAR04896_PR_RPE':[(-1.59,-1.59),(-1000,1000)]})

light_new_labels = create_permutation_dicts({'oxygen':['limited oxygen (2.61)'],\
                      'glucose':['limited glucose (1.93)'],\
                      'lactate':['lactate efflux > 2.95'],\
                      'oxygen RPE -> PR':['fixed (2.31)','unlimited']})
light_new_estimates = create_permutation_dicts({'MAR09048_RPE':[(-2.61,1000)],\
                      'MAR09034_RPE':[ (-1.93,1000)],\
                      'MAR09135_RPE':[ (2.95,1000)],\
                      'MAR04896_PR_RPE':[(-2.31,-2.31),(-1000,1000)]})

In [ ]:
# FVA



In [ ]:
# SAMPLE